In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
import time
start_time = time.time()

In [38]:
Mshs_may = pd.read_csv('MSHS Jan - May 2020 06192020.csv') #Read data

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
Mshs_may = Mshs_may.loc[:, ~Mshs_may.columns.str.contains('^Unnamed')]
Mshs_may.dropna(axis=0,how='all',inplace=True)
Mshs_may=Mshs_may.reset_index(drop=True)

In [40]:
Mshs_may['ItemDepartmentName'].nunique() 

52

In [41]:
data = Mshs_may['Item Description'].apply(str)

In [42]:
data.dtype

dtype('O')

In [43]:
import re
data= data.map(lambda x: re.sub(r'\W+', ' ', x))
data.to_list

<bound method IndexOpsMixin.tolist of 0            PAPER COPY WHITE TOP 10 CA
1        CREAMER COFMATE LIQ ORIG 180CT
2        COCOA MILK CHOCOLATE KCUP 24BX
3                   POWDER BABY AEROSOL
4         RENUZIT SUPER ODOR KILLER 7 5
5             BINDER D RING 2 VUE WHITE
6        REFILL FRSHMTIC AIR WICK LVNDR
7              CLEANER SWIFFER WET 12BX
8               MOP SWFR WETJET STARTER
9        REFILL PDS SWIFFER WETJET 24CT
10           PAPER COPY WHITE TOP 10 CA
11       PAPER COPY RECYCLED 3HP 8 5X11
12       CREAMER COFMATE LIQ ORIG 180CT
13       COCOA MILK CHOCOLATE KCUP 24BX
14                 GMCR TEA GREEN K CUP
15       COFFEE KCUP FLGRS CLASSIC 24BX
16                TONER HP MPS 508X YLW
17                TONER HP MPS 508X MAG
18               TONER HP MPS 508X CYAN
19                TONER HP MPS 508X BLK
20                CUTTER BOX 12PK WHITE
21                CUTTER BOX 12PK WHITE
22                CUTTER BOX 12PK WHITE
23       WATER BTL NSTL PURE LIFE 24 CS
24

In [44]:
#Import stopwords
import nltk
import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("n't")
stopwords.append("br")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
#print (stopwords)

We use 183 stop-words from nltk library.


[nltk_data] Downloading package punkt to /Users/xixikang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xixikang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
# tokenization and stemming
from nltk.stem.snowball import SnowballStemmer
# REGULAR EXPRESSION

stemmer = SnowballStemmer("english")

def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())
            #print(tokens)
    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
            #print(filtered_tokens)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [46]:
#TF-IDF
tfidf_model = TfidfVectorizer(stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,2))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 49472 reviews and 26392 terms.


In [47]:
tfidf_matrix.shape

(49472, 26392)

In [48]:
tf_selected_words = tfidf_model.get_feature_names()

In [49]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [50]:
cosine_sim[0]

array([1.        , 0.        , 0.        , ..., 0.06598838, 0.0550159 ,
       0.        ])

### Bag of  Words

In [51]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(data)
bow_matrix.shape

(49472, 7811)

### Word Embedding

In [52]:
#Tokenization
from nltk.tokenize import sent_tokenize, word_tokenize
data1 = [] 
for i in sent_tokenize(str(data)): 
    temp = [] 
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data1.append(temp)

In [53]:
#CBOW (Continuous Bag of Words)
import gensim 
from gensim.models import Word2Vec 
#the model is loaded. It can be used to perform all of the tasks mentioned above.

# getting word vectors of a word
model1 = gensim.models.Word2Vec(data, min_count = 1,  
                              size = 100, window = 5)


In [54]:
#Skip Gram
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 100, 
                                             window = 5, sg = 1) 

### Cosine Similarity

In [20]:
# Cosine Similarity between words
from sklearn.metrics.pairwise import  cosine_similarity
def cosine_sim_vectors(vec1,vec2):
    vec1 = vec1.reshape(1,-1)
    vec2 = vec2.reshape(1,-1)
    return cosine_similarity(vec1,vec2)[0][0]

In [21]:
# Cosine Similarity between sentences
from collections import Counter
import math

counterA = Counter(data[1])
counterB = Counter(data[3])

def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

return counter_cosine_similarity(counterA,counterB)*100

62.62945776223706

### Training dataset split

In [22]:
y = Mshs_may['ItemDepartmentName'].apply(str)
y.shape

(49472,)

In [23]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.30, random_state=42)

### Support Vector Machines

In [24]:
from sklearn.svm import SVC
svclssifier = SVC(kernel='linear')
svclssifier.fit(X_train,y_train)

y_pred = svclssifier.predict(X_test)

In [25]:
from sklearn.metrics import classification_report, confusion_matrix
confusion_matrix(y_test,y_pred)
#classification_report(y_test,y_pred)
#sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)

array([[   4,    0,    0, ...,    0,    0,    0],
       [   0,    1,    0, ...,    0,    0,    0],
       [   0,    0,  655, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,  976,    0,    0],
       [   0,    0,    0, ...,    0,    4,    0],
       [   0,    0,    0, ...,    0,    0, 1666]])

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9755423797331896

### Naive Bayes Classifier

In [27]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,y_train)
y_pred1 = clf.predict(X_test)

In [28]:
confusion_matrix(y_test,y_pred1)
#classification_report(y_test,y_pred1)

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    2],
       [   0,    0,  658, ...,    0,    0,    1],
       ...,
       [   0,    0,    0, ...,  977,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0, 1673]])

In [29]:
accuracy_score(y_test, y_pred1)

0.9342406683735346

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression

# K Nearest Neighbors
classifier_KNN = KNeighborsClassifier()

# Random Forest
classifier_RF = RandomForestClassifier()

### Random Forest

In [31]:
classifier_RF.fit(X_train, y_train)
classifier_RF.predict(X_test)
classifier_RF.score(X_test, y_test)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9716345505996496

### KNN

In [32]:
classifier_KNN.fit(X_train, y_train)
classifier_KNN.predict(X_test)
classifier_KNN.score(X_test, y_test)

0.9574855140816602

### Cross Validation---Model Selection

In [33]:
model_names = ['Naive Bayes','SVM','Random Forest','KNN']
model_list = [clf, svclssifier,classifier_RF, classifier_KNN]
count = 0

for classifier in model_list:
    cv_score = model_selection.cross_val_score(classifier, X_train, y_train, cv=5)
    print(cv_score)
    print('Model accuracy of ' + model_names[count] + ' is ' + str(cv_score.mean()))
    count += 1

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[0.9233871  0.92891132 0.92562103 0.92991329 0.93180831]
Model accuracy of Naive Bayes is 0.9279282099366647


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[0.97379032 0.97231435 0.97169266 0.97528902 0.98002027]
Model accuracy of SVM is 0.9746213239853574


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[0.96990207 0.9666907  0.96952629 0.97239884 0.97408426]
Model accuracy of Random Forest is 0.9705204329482389


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[0.95722926 0.95270368 0.95060659 0.95592486 0.95555234]
Model accuracy of KNN is 0.9544033438322076


### Grid Search for Parameter Tunning

In [34]:
from sklearn.model_selection import GridSearchCV

# helper function for printing out grid search results 
def print_grid_search_metrics(gs):
    print ("Best score: " + str(gs.best_score_))
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(best_parameters.keys()):
        print(param_name + ':' + str(best_parameters[param_name]))

In [35]:
# Possible hyperparamter options for Random Forest
# Choose the number of trees
parameters = {
    'max_depth': [10, 20, 30, 40, None],
    'max_features': ['auto', 'sqrt'],
    'n_estimators' : [20,40,60,80,100]
}
Grid_RF = GridSearchCV(RandomForestClassifier(),parameters, cv=5)
Grid_RF.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-35-79901d62a44f>", line 11, in <module>
    Grid_RF.fit(X_train, y_train)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py", line 722, in fit
    self._run_search(evaluate_candidates)
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py", line 1191, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py", line 711, in evaluate_candidates
    cv.split(X, y, groups)))
  File "/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 920, in __call__
    while self.dispatch_one_batch(iterator):
  File "/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/parallel.py", line 759, in di

KeyboardInterrupt: 

In [ ]:
print_grid_search_metrics(Grid_RF)
best_RF_model = Grid_RF.best_estimator_

In [ ]:
# Possible hyperparamter options for SVM
# Choose the number of trees
parameters = {
    'kernels ' : ('rbf', 'poly','linear'),
    'gammas': (0.1, 1, 10, 100),
    'C': (0.1, 1, 10, 100, 1000),
    'degrees': (0, 1, 2, 3, 4, 5, 6)   
}
Grid_SVM = GridSearchCV(svclssifier(),parameters, cv=5)
Grid_SVM.fit(X_train, y_train)

In [ ]:
print_grid_search_metrics(Grid_SVM)
best_SVM_model = Grid_SVM.best_estimator_

In [ ]:
# Possible hyperparamter options for KNN
parameters = {
    'leaf_size ' : list(range(1,50)),
    'n_neighbors':list(range(1,50)),
    'p': [1,2]
}
Grid_KNN = GridSearchCV(classifier_KNN(),parameters, cv=5)
Grid_KNN.fit(X_train, y_train)

In [ ]:
print_grid_search_metrics(Grid_KNN)
best_KNN_model = Grid_KNN.best_estimator_

### Evaluation

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# calculate accuracy, precision and recall, [[tn, fp],[]]
def cal_evaluation(classifier, cm):
    tn = cm[0][0]
    fp = cm[0][1]
    fn = cm[1][0]
    tp = cm[1][1]
    accuracy  = (tp + tn) / (tp + fp + fn + tn + 0.0)
    precision = tp / (tp + fp + 0.0)
    recall = tp / (tp + fn + 0.0)
    print (classifier)
    print ("Accuracy is: " + str(accuracy))
    print ("precision is: " + str(precision))
    print ("recall is: " + str(recall))
    print ()

# print out confusion matrices
def draw_confusion_matrices(confusion_matricies):
    class_names = ['Not','Churn']
    for cm in confusion_matrices:
        classifier, cm = cm[0], cm[1]
        cal_evaluation(classifier, cm)

In [ ]:
# Confusion matrix, accuracy, precison and recall for random forest and logistic regression
confusion_matrices = [
    ("Random Forest", confusion_matrix(y_test,best_RF_model.predict(X_test))),
    ("SVM", confusion_matrix(y_test,best_SVM_model.predict(X_test))),
    ("K nearest neighbor", confusion_matrix(y_test, best_KNN_model.predict(X_test)))
]

draw_confusion_matrices(confusion_matrices)

### ROC curve and AUC score

### Random Forest

In [ ]:
from sklearn.metrics import roc_curve
from sklearn import metrics

# Use predict_proba to get the probability results of Random Forest
y_pred_rf = best_RF_model.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, thresh = roc_curve(y_test, y_pred_rf)

In [ ]:
best_RF_model.predict_proba(X_test)

In [ ]:
# ROC curve of Random Forest result
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_rf, tpr_rf, label='RF')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve - RF model')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn import metrics

# AUC score
metrics.auc(fpr_rf,tpr_rf)

#### KNN

In [ ]:
from sklearn.metrics import roc_curve
from sklearn import metrics

# Use predict_proba to get the probability results of Random Forest
y_pred_KNN = best_KNN_model.predict_proba(X_test)[:, 1]
fpr_KNN, tpr_KNN, thresh = roc_curve(y_test, y_pred_rf)

In [ ]:
best_KNN_model.predict_proba(X_test)

In [ ]:
# ROC curve of Random Forest result
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_KNN, tpr_KNN, label='KNN')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve - KNN model')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn import metrics

# AUC score
metrics.auc(fpr_KNN, tpr_KNN)

In [ ]:
print(datetime.datetime.now() - begin_time)

In [ ]:
print("%f seconds" % (time.time() - start_time))